In [ ]:
pip install tensorflow opencv-python-headless


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load Pascal VOC dataset
dataset, info = tfds.load("voc/2007", with_info=True)
train_dataset = dataset['train']
val_dataset = dataset['validation']


In [ ]:
IMG_SIZE = 224

def preprocess_data(data):
    image = tf.image.resize(data['image'], (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize to [0, 1]
    label = data['objects']['label']
    bbox = data['objects']['bbox']
    return image, {'label': label, 'bbox': bbox}

train_dataset = train_dataset.map(preprocess_data)
val_dataset = val_dataset.map(preprocess_data)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# Bounding box output
bbox_output = Dense(4, activation='sigmoid', name='bbox')(x)

# Classification output
class_output = Dense(info.features['objects']['label'].num_classes, activation='softmax', name='label')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=[bbox_output, class_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'bbox': 'mse', 'label': 'sparse_categorical_crossentropy'},
              metrics={'bbox': 'mse', 'label': 'accuracy'})


In [ ]:
# Set batch size and epochs
BATCH_SIZE = 32
EPOCHS = 20

# Prepare the training and validation datasets
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=EPOCHS)


In [ ]:
# Unfreeze some layers in the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss={'bbox': 'mse', 'label': 'sparse_categorical_crossentropy'},
              metrics={'bbox': 'mse', 'label': 'accuracy'})

# Continue training
history_fine = model.fit(train_dataset,
                         validation_data=val_dataset,
                         epochs=EPOCHS)
